# Geocodificar usuarios [Base completa]

In [1]:
import pandas as pd
import get_centroides
from get_centroides import getFromDict
import sqlalchemy
import warnings
import pickle

In [2]:
df = pd.read_csv('./HI_usuarios_completos.csv',  encoding='latin-1') # Datos institucion
df.head()

,anio_ing,nacionalidad,provincia_part,loc_part,cp_part
0,2014,AR,CF,CABA,1416
1,2014,AR,CF,CABA,1416
2,2014,AR,BA,HURLINGHAM,1686
3,2014,AR,BA,ZARATE,2800
4,2014,AR,CF,CABA,1425


In [3]:
df.dtypes

anio_ing           int64
nacionalidad      object
provincia_part    object
loc_part          object
cp_part           object
dtype: object

In [4]:
df.nacionalidad.unique() # Algunos NaN y fechas en el campo nacionalidad.

array(['AR', 'CA', 'UY', 'PY', 'IT', 'US', 'AU', 'BR', 'JP', 'BO', 'CN',
       'ES', 'CO', 'VE', 'KR', 'EC', 'PE', 'CL', 'FR', 'GB', 'HL', 'PT',
       'CU', 'SY', 'KP', 'PA', 'CR', 'YU', 'HN', 'PL', 'UA', 'DE', 'MX',
       'GT', 'IL', 'HT', 'AO', 'RU', '30/06/2015', 'IO', 'CH', 'HR', 'AT',
       'LT', 'HU', 'TH', 'ET', '05/08/1987', 'DO', 'IN', 'TW', 'MK', 'SV',
       'NZ', 'GR', 'AM', 'DM', 'SK', 'PH', 'SE', 'BG', 'VN', 'DK', 'SI',
       'NI', 'IR', 'FI', 'IE', 'KZ', 'TR', 'LI', nan, 'BE', 'LB', 'BY',
       'IS', 'ZA', 'NG', 'MZ', 'BD', 'PS'], dtype=object)

In [5]:
df.provincia_part.unique() # Parece ser que provincia es un campo estructurado

array(['CF', 'BA', nan, 'LP', 'MZ', 'SL', 'CA', 'SC', 'CH', 'RN', 'TF',
       'SF', 'ER', 'MI', 'CB', 'TU', 'NE', 'CO', 'JU', 'SJ', 'SA', 'FO',
       'SE', 'CT'], dtype=object)

In [6]:
df.nacionalidad.str.contains('/').sum() # Tengo el mismo problema que antes

2

In [7]:
df.isna().sum() 
# 206 usuarios sin  nacionalidad
# 4820 sin provincia reportada
# 1 solo sin CP    

anio_ing             0
nacionalidad       206
provincia_part    4820
loc_part           263
cp_part              1
dtype: int64

Pareceria que todo es muy similar al dataset mas pequeño con el que trabajamos antes. Exploraremos algunas aspectos mas granulares (ej por nacionalidad) parar ver si en los casos mas 'anomalos' hay algun conflicto.

In [8]:
exp_gb = df.groupby(['anio_ing','nacionalidad','provincia_part']).agg('count').reset_index()

In [9]:
exp_gb.isna().sum() # Los datos de gente de otra nacionalidad parecerian estar todos completos

anio_ing          0
nacionalidad      0
provincia_part    0
loc_part          0
cp_part           0
dtype: int64

In [10]:
exp_gb[exp_gb.nacionalidad != 'AR'].provincia_part.unique() 
# Parece que todos las personas que reportan otra nacionalidad igualmente seleccionan una provincia
# en el territorio argentino

array(['BA', 'CF', 'SA', 'RN', 'CO', 'MI', 'CA'], dtype=object)

## Preprocessing

Vamos a remover los dos sujetos que presentan fechas en las nacionalidades. 

In [11]:
# Vamos a asumir que los campos con valor None en nacionalidad se deben a errores de los usuarios, pasaran a ser `desconocido`.
# Los campos que tengan fechas en la nacionalidad seran removidos.
df.nacionalidad[df.nacionalidad.isna()] = 'DESCONOCIDO'

C:\Users\joaqu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
df_clean = df.loc[df.nacionalidad.str.contains('/') == False].reset_index(drop = True).copy()

In [13]:
df_clean.shape[0] - df.shape[0] # Eliminamos dos usuarios de 80k

-2

Para este caso asignare un `user_id` simulando un dataset real. 

In [14]:
df_clean['user_id'] = list(map(lambda x: 'ID_' + str(x), df_clean.index.to_list()))

In [15]:
# Poner loc-part todo en mayuscula
df_clean.loc_part = df_clean.loc_part.apply(lambda x: x.upper() if isinstance(x,str) else None)  

## Acceder a la base de datos 

In [16]:
warnings.simplefilter("ignore", category=sqlalchemy.exc.SAWarning) # ignor sqlalchemy warnings
engine = sqlalchemy.create_engine('sqlite:///SQL/CPA.db')
con = engine.connect()
cpa_prov = pd.read_sql('provincia', con=con) # provincia por codigo postal
cpa_local = pd.read_sql('localidad', con=con) # localidad por codigo postal
#Se debe remover este campo del dataframe antes de realizar el join.
cpa_local.drop(cpa_local[cpa_local.codigopostal == 0].index).reset_index(drop = True, inplace=True)
cpa_capital =  pd.read_sql('comunas_capital', con=con) # localidades y comuna de capital por CP

In [17]:
cpa_provincia_completo = cpa_local.merge(cpa_prov, left_on='provincia_id', right_on='p_id',
                                         suffixes=['_localidad','_provincia']).drop(columns = 'p_id')
CPA_ARGENTINA =  cpa_provincia_completo.merge(cpa_capital, on= 'provincia_id' ,
                                              how='left' , suffixes = ('_x',''))  
# El dataframe mas completo hasta el momento de CPA y localidad
# Unir los dos codigos postales de las distintas tablas
CPA_ARGENTINA['codigopostal'] = CPA_ARGENTINA.codigopostal.fillna(CPA_ARGENTINA.codigopostal_x)
CPA_ARGENTINA.drop(columns='codigopostal_x', inplace= True) # eliminar la columna CP repetida
CPA_ARGENTINA['codigopostal'] = CPA_ARGENTINA.codigopostal.astype(int) # Transformar a integer el CP

In [18]:
set_provincia = set(cpa_provincia_completo.codigopostal)
set_provincia.remove(0) # Nuevamente removemos el 0 del codigo postal de provincia
set_capital = set(cpa_capital.codigopostal)
set_capital.intersection(set_provincia) # No hay CP en capital que se repitan en las otras provincias

set()

In [19]:
## Acceder a los datos de la API
departamentos = pd.read_csv('https://apis.datos.gob.ar/georef/api/departamentos.csv')
localidades = pd.read_csv('https://apis.datos.gob.ar/georef/api/localidades.csv')
municipios = pd.read_csv('https://apis.datos.gob.ar/georef/api/municipios.csv')
provincias = pd.read_csv('https://apis.datos.gob.ar/georef/api/provincias.csv')

## Populando nueva tabla con datos de geolocalizacion
### Asignar localidad, departamento(comuna CP) y provincia usando  solo CP

In [20]:
# Listado de CP que no son unicos
provincia_por_cp = cpa_provincia_completo.groupby('codigopostal')['nombre_provincia'].nunique()
cp_prov_no_unicos = set(provincia_por_cp.loc[provincia_por_cp > 1].index)

localidad_por_cp = cpa_provincia_completo.groupby('codigopostal')['nombre_localidad'].nunique()
cp_localidad_no_unicos = set(localidad_por_cp.loc[localidad_por_cp > 1].index) 

capital_barrio_por_cp = cpa_capital.groupby('codigopostal')['barrio'].nunique()
capital_cp_barrio_no_unicos = set(capital_barrio_por_cp.loc[capital_barrio_por_cp > 1].index)

capital_comuna_por_cp = cpa_capital.groupby('codigopostal')['comuna'].nunique()
capital_cp_comuna_no_unicos = set(capital_comuna_por_cp.loc[capital_comuna_por_cp > 1].index)

In [21]:
# Funciones utiles
def infer_provincia(cp):
    try:
        int(cp)
    except:
        return 'CP INDEFINIDO'
    if int(cp) in set_capital:
        return 'CIUDAD AUTONOMA DE BUENOS AIRES'
    if int(cp) in set_provincia:
        if  int(cp) in cp_prov_no_unicos:
            return  'NO UNICA PROVINCIA'
        else:
            prov = cpa_provincia_completo.loc[cpa_provincia_completo.codigopostal == int(cp)].nombre_provincia.unique()
            return prov[0].upper()
    if (int(cp) not in set_provincia and int(cp) not in set_capital):
        return 'CP INDEFINIDO'
    if cp.isna() or int(cp) == 0:
        return 'CP INDEFINIDO'

def infer_localidad(cp):
    try:
        int(cp)
    except:
        return 'CP INDEFINIDO'
    cp = int(cp)
    if cp in set_capital:
        if cp in capital_cp_barrio_no_unicos:
                return 'NO UNICA LOCALIDAD'
        else:
            localidad = cpa_capital.loc[cpa_capital.codigopostal == cp].barrio.unique()[0]
            return localidad.upper()
    if cp in set_provincia:
        if cp in cp_localidad_no_unicos:
            return 'NO UNICA LOCALIDAD'
        else:
            localidad = cpa_provincia_completo.loc[cpa_provincia_completo.codigopostal == cp].nombre_localidad.unique()[0]
            return localidad.upper()
    if cp not in set_provincia and cp not in set_capital:
        return 'CP INDEFINIDO'
           
def infer_departamento(cp):
    try:
        int(cp)
    except:
        return 'CP INDEFINIDO'
    cp = int(cp)
    if cp in set_capital:
        if cp in capital_cp_barrio_no_unicos:
            return 'NO UNICO DEPARTAMENTO'
        else:
            departamento = cpa_capital.loc[cpa_capital.codigopostal == cp].comuna.unique()
            return f'COMUNA {departamento[0]}'

In [22]:
augmented_df = df_clean.copy() # este es el dataframe que vamos a popular
#Agregamos columnas
columnas =  ['provincia_por_cp' ,'localidad_por_cp' ,'departamento_por_cp' ,'localidad_por_API', 'localidad_id_API', 'municipio_por_API',
             'municipio_id_API', 'departamento_por_API', 'departamento_id_API', 'provincia_por_API', 'provincia_id_API','centroide_lat_localidad',
             'centroide_lon_localidad', 'centroide_lat_municipio','centroide_lon_municipio','centroide_lat_departamento','centroide_lon_departamento',
             'centroide_lat_provincia','centroide_lon_provincia']
for i in columnas:
    augmented_df[i] = None

In [23]:
# Provincia
augmented_df['provincia_por_cp'] = augmented_df.cp_part.apply(infer_provincia)
# Localidad
augmented_df['localidad_por_cp'] = augmented_df.cp_part.apply(infer_localidad)
# Departamento 'Comuna'
augmented_df['departamento_por_cp'] = augmented_df.cp_part.apply(infer_departamento)

In [24]:
 augmented_df.sample(10)

,anio_ing,nacionalidad,provincia_part,loc_part,cp_part,user_id,provincia_por_cp,localidad_por_cp,departamento_por_cp,localidad_por_API,...,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
1010,2014,AR,BA,JOSE CLEMENTE PAZ,1665,ID_1010,BUENOS AIRES,JOSE CLEMENTE PAZ,None,None,...,None,None,None,None,None,None,None,None,None,None
37724,2016,AR,BA,VALENTIN ALSINA,1822,ID_37724,BUENOS AIRES,VALENTIN ALSINA,None,None,...,None,None,None,None,None,None,None,None,None,None
27937,2016,AR,BA,OLIVOS,1636,ID_27937,BUENOS AIRES,NO UNICA LOCALIDAD,None,None,...,None,None,None,None,None,None,None,None,None,None
62741,2018,AR,CF,CABA,1429,ID_62741,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,None,...,None,None,None,None,None,None,None,None,None,None
77786,2019,AR,CF,CDAD AUTONOMA BS AS,1200,ID_77786,CP INDEFINIDO,CP INDEFINIDO,None,None,...,None,None,None,None,None,None,None,None,None,None
52951,2017,AR,BA,OLIVOS,1602,ID_52951,BUENOS AIRES,NO UNICA LOCALIDAD,None,None,...,None,None,None,None,None,None,None,None,None,None
11995,2014,AR,BA,VILLA BALLESTER,1653,ID_11995,BUENOS AIRES,VILLA BALLESTER,None,None,...,None,None,None,None,None,None,None,None,None,None
48679,2017,AR,BA,SAN ANDRES,1651,ID_48679,BUENOS AIRES,SAN ANDRES,None,None,...,None,None,None,None,None,None,None,None,None,None
83468,2019,AR,BA,GREGORIO DE LA FERR,1757,ID_83468,BUENOS AIRES,GREGORIO DE LAFERRERE,None,None,...,None,None,None,None,None,None,None,None,None,None
66626,2018,AR,BA,CASEROS,1678,ID_66626,BUENOS AIRES,CASEROS,None,None,...,None,None,None,None,None,None,None,None,None,None


### Conseguir datos de los que tienen provincia y localidad provista por codigo postal

In [25]:
# Seleccionamos solo las prov con loc
gby_prov_loc = augmented_df.groupby(['provincia_por_cp','localidad_por_cp']).count().reset_index(level=['provincia_por_cp', 'localidad_por_cp']).copy()
gby_prov_loc = gby_prov_loc.loc[(gby_prov_loc.localidad_por_cp != 'NO UNICA LOCALIDAD') & (gby_prov_loc.localidad_por_cp != 'CP INDEFINIDO')]

In [26]:
gby_prov_loc.drop(columns = ['anio_ing','user_id','nacionalidad','provincia_part','cp_part','loc_part','departamento_por_cp'], inplace= True)
gby_prov_loc.loc[:,'localidad_por_API' : 'centroide_lon_provincia'] = None # inicializo todos los campos a zero
print(gby_prov_loc.shape)
gby_prov_loc.head(4) # Tenemos 105 combinacinos unicas no identificadas

(105, 18)


,provincia_por_cp,localidad_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
0,BUENOS AIRES,9 DE ABRIL,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,BUENOS AIRES,AEROPUERTO EZEIZA,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,BUENOS AIRES,ALEJANDRO KORN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,BUENOS AIRES,BANFIELD,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [27]:
# Crea una lista de tuplas para realizar el post. Para mas info consultar get_centroides.py
prov_loc = list(gby_prov_loc.apply(lambda x: (x.provincia_por_cp, x.localidad_por_cp), axis=1))
# Manda el POST request
datos = get_centroides.post_centroides('localidades', prov_loc, prov = True) # Consulta get_centroides.py

In [28]:
#funcion para rellenar los df
def rellenar_df(df,datos):
    df['provincia_por_API'] = [getFromDict(i,'provincia','nombre') for i in datos]
    df['provincia_id_API'] = [getFromDict(i,'provincia','id') for i in datos]
    df['departamento_por_API'] = [getFromDict(i,'departamento','nombre') for i in datos]
    df['departamento_id_API'] = [getFromDict(i,'departamento','id') for i in datos]
    df['municipio_por_API'] = [getFromDict(i,'municipio','nombre') for i in datos]
    df['municipio_id_API'] = [getFromDict(i,'municipio','id') for i in datos]
    df['localidad_por_API'] = [getFromDict(i,'nombre') for i in datos]
    df['localidad_id_API'] = [getFromDict(i,'id') for i in datos]
    df['centroide_lat_localidad'] = [getFromDict(i,'centroide','lat') for i in datos]
    df['centroide_lon_localidad'] = [getFromDict(i,'centroide','lon') for i in datos]
    cols_s = ['municipio_por_API', 'departamento_por_API', 'provincia_por_API']
    cols_i = ['departamento_id_API','municipio_id_API','provincia_id_API']
    # To upper
    for i in cols_s:
        df[i] = df[i].apply(lambda x: x.upper() if isinstance(x,str) else 'NO HUBO REQUEST')
    for i in cols_i:
        df[i] = df[i].apply(lambda x: (int(x) if  (x != 'NO HUBO REQUEST') and pd.notna(x) else x)) 
    return df

In [29]:
gby_prov_loc = rellenar_df(gby_prov_loc,datos)
gby_prov_loc.sample(5)

,provincia_por_cp,localidad_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
104,CIUDAD AUTONOMA DE BUENOS AIRES,SAN NICOLÁS,SAN NICOLAS,02007010005,NO HUBO REQUEST,None,COMUNA 1,2007,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6037,-58.3805,None,None,None,None,None,None
0,BUENOS AIRES,9 DE ABRIL,9 DE ABRIL,06260010005,ESTEBAN ECHEVERRÍA,60260,ESTEBAN ECHEVERRÍA,6260,BUENOS AIRES,6,-34.7565,-58.4903,None,None,None,None,None,None
67,BUENOS AIRES,SIERRA DE LA VENTANA,SIERRA DE LA VENTANA,06819030000,TORNQUIST,60819,TORNQUIST,6819,BUENOS AIRES,6,-38.1367,-61.7957,None,None,None,None,None,None
29,BUENOS AIRES,ISLA MARTIN GARCIA,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,None,None,None,None,None,None
40,BUENOS AIRES,LONGCHAMPS,LONGCHAMPS,06028010007,ALMIRANTE BROWN,60028,ALMIRANTE BROWN,6028,BUENOS AIRES,6,-34.8609,-58.3891,None,None,None,None,None,None


In [30]:
gby_prov_loc['copy_index'] = gby_prov_loc.index

#Asignar centroide de municipios
gby_prov_loc.municipio_id_API = gby_prov_loc.municipio_id_API.apply(lambda x: (int(x) if  (x != 'NO HUBO REQUEST') and pd.notna(x) else x)) # Pasar a integer la columna
gby_prov_loc.centroide_lat_municipio = gby_prov_loc.merge(municipios, left_on= 'municipio_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lat']
gby_prov_loc.centroide_lon_municipio = gby_prov_loc.merge(municipios, left_on= 'municipio_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lon']

#Asignar centroide de departamentos
gby_prov_loc.departamento_id_API = gby_prov_loc.departamento_id_API.apply(lambda x: (int(x) if  x != 'NO HUBO REQUEST' else x)) # Pasar a integer la columna
gby_prov_loc.centroide_lat_departamento = gby_prov_loc.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lat']
gby_prov_loc.centroide_lon_departamento = gby_prov_loc.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lon']

#Asignar centroide de provincias
gby_prov_loc.provincia_id_API = gby_prov_loc.provincia_id_API.apply(lambda x: (int(x) if  x != 'NO HUBO REQUEST' else x)) # Pasar a integer la columna
gby_prov_loc.centroide_lat_provincia = gby_prov_loc.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lat']
gby_prov_loc.centroide_lon_provincia = gby_prov_loc.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lon']

In [31]:
gby_prov_loc.sample(5)

,provincia_por_cp,localidad_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,copy_index
10,BUENOS AIRES,CASEROS,CASEROS,06840010001,TRES DE FEBRERO,60840,TRES DE FEBRERO,6840,BUENOS AIRES,6,-34.6089,-58.5637,-34.595980,-58.579093,-34.595980,-58.579093,-36.676942,-60.558832,10
31,BUENOS AIRES,JOSE CLEMENTE PAZ,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NaN,NaN,31
60,BUENOS AIRES,SAN BERNARDO DEL TUYU,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NaN,NaN,60
82,BUENOS AIRES,VILLA SANTOS TESEI,VILLA SANTOS TESEI,06408010002,HURLINGHAM,60408,HURLINGHAM,6408,BUENOS AIRES,6,-34.6124,-58.6538,-34.599214,-58.649694,-34.599214,-58.649694,-36.676942,-60.558832,82
95,CIUDAD AUTONOMA DE BUENOS AIRES,LA BOCA,BOCA,02028010002,NO HUBO REQUEST,None,COMUNA 4,2028,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6311,-58.3568,NaN,NaN,-34.642079,-58.387455,-34.614493,-58.445856,95


#### Agregar los datos a la tabla aumentada

In [32]:
pd.set_option('display.max_columns', 50)  # Para ver todas las filas
# Vamos a hacer un Join con los valores que tengan la misma provincia y localidad por CP
augmented_df['copy_index'] = augmented_df.index.copy()  # Guardar los index iniciales
gby_prov_loc.drop(columns= 'copy_index', inplace= True)

augmented_df = augmented_df.merge(gby_prov_loc , on = ['provincia_por_cp','localidad_por_cp'],
                                  how= 'left', suffixes= ['_del','']).set_index(augmented_df['copy_index']).copy()
                                                                                 
augmented_df = augmented_df.drop(columns = augmented_df.filter(regex = '._del').columns)
augmented_df.drop(columns = ['copy_index'], inplace = True)

In [33]:
augmented_df.sample(5)

,anio_ing,nacionalidad,provincia_part,loc_part,cp_part,user_id,provincia_por_cp,localidad_por_cp,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
copy_index,,,,,,,,,,,,,,,,,,,,,,,,,
302,2014,AR,BA,FLORIDA,1602,ID_302,BUENOS AIRES,NO UNICA LOCALIDAD,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69058,2019,AR,BA,ITUZAINGO,1714,ID_69058,BUENOS AIRES,ITUZAINGO,None,ITUZAINGO CENTRO,06410010001,ITUZAINGÓ,60410,ITUZAINGÓ,6410,BUENOS AIRES,6,-34.6441,-58.6885,-34.636002,-58.688758,-34.636002,-58.688758,-36.676942,-60.558832
34278,2016,AR,CF,CABA,1424,ID_34278,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61012,2018,AR,BA,SAN ISIDRO,1642,ID_61012,BUENOS AIRES,SAN ISIDRO,None,SAN ISIDRO,06756010005,SAN ISIDRO,60756,SAN ISIDRO,6756,BUENOS AIRES,6,-34.477,-58.5272,-34.486950,-58.537265,-34.486950,-58.537265,-36.676942,-60.558832
71034,2018,AR,BA,LA TABLADA,1766,ID_71034,BUENOS AIRES,TABLADA,None,LA TABLADA,06427010006,LA MATANZA,60427,LA MATANZA,6427,BUENOS AIRES,6,-34.6871,-58.5256,-34.770165,-58.625449,-34.770165,-58.625449,-36.676942,-60.558832


### Conseguir datos de los que tienen solo provincia por codigo postal

In [34]:
gby_solo_provincia = augmented_df.groupby(['loc_part', 'provincia_por_cp','localidad_por_cp']).count(
                        ).reset_index(['loc_part','provincia_por_cp','localidad_por_cp']) # con este subset armar un post request
# Selecciono las opciones en las cuales el CP no dio una localidad definida pero si me dio provincia
gby_solo_provincia = gby_solo_provincia.loc[(gby_solo_provincia.provincia_por_cp != 'CP INDEFINIDO') &
                                            (gby_solo_provincia.provincia_por_cp != 'NO UNICA PROVINCIA') &
                                            (gby_solo_provincia.localidad_por_cp == 'NO UNICA LOCALIDAD')]
print(gby_solo_provincia.shape)
gby_solo_provincia.head(5)

(534, 25)


,loc_part,provincia_por_cp,localidad_por_cp,anio_ing,nacionalidad,provincia_part,cp_part,user_id,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
0,VENEZUELA Y PERU,BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,-1,BUENOS AIRES,NO UNICA LOCALIDAD,22,22,22,22,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,-1,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,2,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,-1,SAN JUAN,NO UNICA LOCALIDAD,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,-1,SAN LUIS,NO UNICA LOCALIDAD,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
# Crea una lista de tuplas para realizar el post
prov_loc = list(gby_solo_provincia.apply(lambda x: (x.provincia_por_cp, x.loc_part), axis=1))  
#En vez de localidad usamos loc_part (localidad reportada)
# Manda el POST request
datos = get_centroides.post_centroides('localidades', prov_loc, prov = True) # Consulta get_centroides.py

In [36]:
len(datos)

534

In [37]:
# Rellena la tabla con los datos
gby_solo_provincia = rellenar_df(gby_solo_provincia,datos)
gby_solo_provincia.sample(5)

,loc_part,provincia_por_cp,localidad_por_cp,anio_ing,nacionalidad,provincia_part,cp_part,user_id,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
204,CAPITAL,BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,1,1,0,CAPITAN SARMIENTO,06140010000,CAPITÁN SARMIENTO,60140,CAPITÁN SARMIENTO,6140,BUENOS AIRES,6,-34.1724,-59.7893,0,0,0,0,0,0
546,GENERAL BELGRANO,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,1,1,1,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,0,0,0,0,0,0
824,LOMAS DE ZAMORA,BUENOS AIRES,NO UNICA LOCALIDAD,37,37,36,37,37,0,LOMAS DE ZAMORA,06490010003,LOMAS DE ZAMORA,60490,LOMAS DE ZAMORA,6490,BUENOS AIRES,6,-34.7628,-58.4268,0,0,0,0,0,0
13,-1,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,2,2,2,2,2,2,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,0,0,0,0,0,0
739,LA PAZ,ENTRE RÍOS,NO UNICA LOCALIDAD,1,1,1,1,1,0,LA PAZ,30070040000,NO HUBO REQUEST,None,LA PAZ,30070,ENTRE RÍOS,30,-30.7415,-59.6434,0,0,0,0,0,0


In [38]:
gby_solo_provincia['copy_index_2'] = gby_solo_provincia.index

# Asignar centroide de municipios
gby_solo_provincia.centroide_lat_municipio = gby_solo_provincia.merge(municipios, left_on= 'municipio_id_API', right_on='id', how='left' ).set_index(gby_solo_provincia['copy_index_2'])['centroide_lat']
gby_solo_provincia.centroide_lon_municipio = gby_solo_provincia.merge(municipios, left_on= 'municipio_id_API', right_on='id', how='left' ).set_index(gby_solo_provincia['copy_index_2'])['centroide_lon']
# Asignar centroide de departamentos
gby_solo_provincia.centroide_lon_departamento = gby_solo_provincia.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how ='left').set_index(gby_solo_provincia['copy_index_2'])['centroide_lon']
gby_solo_provincia.centroide_lat_departamento = gby_solo_provincia.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how='left').set_index(gby_solo_provincia['copy_index_2'])['centroide_lat']
# Asignar centroide de provincias
gby_solo_provincia.centroide_lat_provincia = gby_solo_provincia.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_solo_provincia['copy_index_2'])['centroide_lat']
gby_solo_provincia.centroide_lon_provincia = gby_solo_provincia.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_solo_provincia['copy_index_2'])['centroide_lon']

In [39]:
gby_solo_provincia.sample(5) 

,loc_part,provincia_por_cp,localidad_por_cp,anio_ing,nacionalidad,provincia_part,cp_part,user_id,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,copy_index_2
445,DOCK SUD,BUENOS AIRES,NO UNICA LOCALIDAD,3,3,2,3,3,0,DOCK SUD,06035010004,AVELLANEDA,60035,AVELLANEDA,6035,BUENOS AIRES,6,-34.6491,-58.3412,-34.678206,-58.341117,-34.678206,-58.341117,-36.676942,-60.558832,445
1486,VIRREYES,BUENOS AIRES,NO UNICA LOCALIDAD,38,38,38,38,38,0,VIRREYES,06749010003,SAN FERNANDO,60749,SAN FERNANDO,6749,BUENOS AIRES,6,-34.4636,-58.5762,-34.151270,-58.545954,-34.151270,-58.545954,-36.676942,-60.558832,1486
739,LA PAZ,ENTRE RÍOS,NO UNICA LOCALIDAD,1,1,1,1,1,0,LA PAZ,30070040000,NO HUBO REQUEST,None,LA PAZ,30070,ENTRE RÍOS,30,-30.7415,-59.6434,NaN,NaN,-30.897456,-59.500449,-32.058874,-59.201448,739
1410,VILLA GENERAL BELGR,CÓRDOBA,NO UNICA LOCALIDAD,1,1,1,1,1,0,VILLA GENERAL BELGRANO,14007310000,VILLA GENERAL BELGRANO,140049,CALAMUCHITA,14007,CÓRDOBA,14,-31.9808,-64.5606,-31.984374,-64.585415,-32.202712,-64.616717,-32.142933,-63.801753,1410
44,ADROGUE-ALTE BROWN,BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,1,1,0,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NaN,NaN,44


###  Conseguir datos de los que no obtuvimos ni provincia ni localidad


In [40]:
gby_solo_loc_part = augmented_df.groupby(['loc_part', 'provincia_por_cp','localidad_por_cp']).count(
                        ).reset_index(['loc_part','provincia_por_cp','localidad_por_cp']) # con este subset armar un post request
# Selecciono las opciones en las cuales el CP no dio una localidad definida, ni provincia o tambien si el CP es indefinido
gby_solo_loc_part = gby_solo_loc_part.loc[(gby_solo_loc_part.provincia_por_cp == 'CP INDEFINIDO') |
                                            (gby_solo_loc_part.provincia_por_cp == 'NO UNICA PROVINCIA') &
                                            (gby_solo_loc_part.localidad_por_cp == 'NO UNICA LOCALIDAD')]
print(gby_solo_loc_part.shape)
gby_solo_loc_part.sample(5) #221 rows unicas

(221, 25)


,loc_part,provincia_por_cp,localidad_por_cp,anio_ing,nacionalidad,provincia_part,cp_part,user_id,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
198,CANNING,CP INDEFINIDO,CP INDEFINIDO,15,15,15,15,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
952,MUNRO,CP INDEFINIDO,CP INDEFINIDO,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
844,LOS POLVORINES,CP INDEFINIDO,CP INDEFINIDO,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1106,RINCON DE MILBERG,CP INDEFINIDO,CP INDEFINIDO,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1109,RIO GRANDE,CP INDEFINIDO,CP INDEFINIDO,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
# Crea una lista de tuplas para realizar el post
prov_loc = list(gby_solo_loc_part.apply(lambda x: (x.provincia_por_cp, x.loc_part), axis=1))  # no necesito saber que no se sabe la localidad
# En vez de localidad usamos loc_part (localidad reportada)
# Manda el POST request
datos = get_centroides.post_centroides('localidades', prov_loc, prov = False) # Consulta get_centroides.py, no uso la provincia en este caso

In [42]:
len(datos)

221

In [43]:
# Rellena la tabla con los datos
gby_solo_loc_part = rellenar_df(gby_solo_loc_part,datos)
gby_solo_loc_part.sample(5)

,loc_part,provincia_por_cp,localidad_por_cp,anio_ing,nacionalidad,provincia_part,cp_part,user_id,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
1127,SAENZ PEÑA,CP INDEFINIDO,CP INDEFINIDO,5,5,5,5,5,0,SAENZ PEÑA,06840010012,TRES DE FEBRERO,60840,TRES DE FEBRERO,6840,BUENOS AIRES,6,-34.5993,-58.5321,0,0,0,0,0,0
687,ITUZAINGO,CP INDEFINIDO,CP INDEFINIDO,37,37,27,37,37,0,ITUZAINGO,18084020000,ITUZAINGÓ,180168,ITUZAINGÓ,18084,CORRIENTES,18,-27.591,-56.704,0,0,0,0,0,0
1021,PILAR,CP INDEFINIDO,CP INDEFINIDO,3,3,3,3,3,0,PILAR,06638040008,PILAR,60638,PILAR,6638,BUENOS AIRES,6,-34.4663,-58.9227,0,0,0,0,0,0
14,-1,CP INDEFINIDO,CP INDEFINIDO,24,24,24,24,24,0,1 DE AGOSTO,14021310004,ESTACIÓN JUÁREZ CELMAN,140105,COLÓN,14021,CÓRDOBA,14,-31.3069,-64.1785,0,0,0,0,0,0
1317,TRONCOS DEL TALAR,CP INDEFINIDO,CP INDEFINIDO,8,8,8,8,8,0,LOS TRONCOS DEL TALAR,06805010007,TIGRE,60805,TIGRE,6805,BUENOS AIRES,6,-34.4426,-58.6174,0,0,0,0,0,0


In [44]:
gby_solo_loc_part['copy_index_2'] = gby_solo_loc_part.index
# Asignar centroide de municipios
gby_solo_loc_part.centroide_lat_municipio = gby_solo_loc_part.merge(municipios, left_on= 'municipio_id_API', right_on='id', how='left' ).set_index(gby_solo_loc_part['copy_index_2'])['centroide_lat']
gby_solo_loc_part.centroide_lon_municipio = gby_solo_loc_part.merge(municipios, left_on= 'municipio_id_API', right_on='id', how='left' ).set_index(gby_solo_loc_part['copy_index_2'])['centroide_lon']
# Asignar centroide de departamentos
gby_solo_loc_part.centroide_lon_departamento = gby_solo_loc_part.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how ='left').set_index(gby_solo_loc_part['copy_index_2'])['centroide_lon']
gby_solo_loc_part.centroide_lat_departamento = gby_solo_loc_part.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how='left').set_index(gby_solo_loc_part['copy_index_2'])['centroide_lat']
# Asignar centroide de provincias
gby_solo_loc_part.centroide_lat_provincia = gby_solo_loc_part.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_solo_loc_part['copy_index_2'])['centroide_lat']
gby_solo_loc_part.centroide_lon_provincia = gby_solo_loc_part.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_solo_loc_part['copy_index_2'])['centroide_lon']
gby_solo_loc_part.sample(10)

,loc_part,provincia_por_cp,localidad_por_cp,anio_ing,nacionalidad,provincia_part,cp_part,user_id,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,copy_index_2
1388,VILLA CAROLA,CP INDEFINIDO,CP INDEFINIDO,3,3,3,3,3,0,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NaN,NaN,1388
1127,SAENZ PEÑA,CP INDEFINIDO,CP INDEFINIDO,5,5,5,5,5,0,SAENZ PEÑA,06840010012,TRES DE FEBRERO,60840,TRES DE FEBRERO,6840,BUENOS AIRES,6,-34.5993,-58.5321,-34.595980,-58.579093,-34.595980,-58.579093,-36.676942,-60.558832,1127
1266,TABLADA,CP INDEFINIDO,CP INDEFINIDO,21,21,14,21,21,0,LA TABLADA,06427010006,LA MATANZA,60427,LA MATANZA,6427,BUENOS AIRES,6,-34.6871,-58.5256,-34.770165,-58.625449,-34.770165,-58.625449,-36.676942,-60.558832,1266
687,ITUZAINGO,CP INDEFINIDO,CP INDEFINIDO,37,37,27,37,37,0,ITUZAINGO,18084020000,ITUZAINGÓ,180168,ITUZAINGÓ,18084,CORRIENTES,18,-27.591,-56.704,-27.967902,-56.917024,-27.911426,-56.790476,-28.774305,-57.801219,687
917,MONTE CHINGOLO,CP INDEFINIDO,CP INDEFINIDO,4,4,4,4,4,0,MONTE CHINGOLO,06434010004,LANÚS,60434,LANÚS,6434,BUENOS AIRES,6,-34.7302,-58.3562,-34.705782,-58.395448,-34.705782,-58.395448,-36.676942,-60.558832,917
1038,POSADAS,NO UNICA PROVINCIA,NO UNICA LOCALIDAD,14,14,14,14,14,0,POSADAS,54028030000,POSADAS,540119,CAPITAL,54028,MISIONES,54,-27.3662,-55.894,-27.433817,-55.958840,-27.550785,-55.856845,-26.875397,-54.651697,1038
1104,RICARDO ROJAS,CP INDEFINIDO,CP INDEFINIDO,2,2,2,2,2,0,RICARDO ROJAS,06805010008,TIGRE,60805,TIGRE,6805,BUENOS AIRES,6,-34.4563,-58.6807,-34.381589,-58.599454,-34.381589,-58.599454,-36.676942,-60.558832,1104
91,AVELLANEDA,CP INDEFINIDO,CP INDEFINIDO,49,49,16,49,49,0,AVELLANEDA,06035010002,AVELLANEDA,60035,AVELLANEDA,6035,BUENOS AIRES,6,-34.6627,-58.3657,-34.678206,-58.341117,-34.678206,-58.341117,-36.676942,-60.558832,91
845,LOTE 86 NEUQUEN CAP.,NO UNICA PROVINCIA,NO UNICA LOCALIDAD,1,1,1,1,1,0,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NaN,NaN,845
1496,WILDE,CP INDEFINIDO,CP INDEFINIDO,3,3,1,3,3,0,WILDE,06035010009,AVELLANEDA,60035,AVELLANEDA,6035,BUENOS AIRES,6,-34.7062,-58.3218,-34.678206,-58.341117,-34.678206,-58.341117,-36.676942,-60.558832,1496


In [45]:
gby_solo_loc_part[gby_solo_loc_part['localidad_por_API'] == 'NO HUBO REQUEST'].shape #34 request no devolvieron resultados

(34, 26)

## Agregar todos los datos a la tabla

In [46]:
aux_df = gby_solo_loc_part.append(gby_solo_provincia, sort = False) #append all tables in one df. Except prov_loc 

In [47]:
# Vamos a hacer un mege con los valores que tengan la misma provincia y localidad por CP y loc_part
augmented_df['copy_index'] = augmented_df.index.copy()  # Guardar los index iniciales

augmented_df2 = augmented_df.merge(aux_df ,
                                   on = ['loc_part','provincia_por_cp','localidad_por_cp'],
                                   how= 'left',
                                   suffixes= ['','_added']).set_index(augmented_df['copy_index']).copy()


In [48]:
def assign_value(row, column = None):
    '''
    Simple function that takes a row and a column and concatenate two columns after permorming a merge
    In this case the right table with the same column name was added a `_added` termination
    '''
    if pd.isna(row[column]):
        return row[column + '_added']
    else:
        return row[column]

In [49]:
col = ['provincia_por_API','localidad_por_API','departamento_por_API','municipio_por_API',
        'centroide_lat_localidad','centroide_lon_localidad','centroide_lat_departamento',
        'centroide_lon_departamento','centroide_lat_municipio','centroide_lon_municipio','centroide_lat_provincia',
        'centroide_lon_provincia','provincia_id_API','departamento_id_API','municipio_id_API','localidad_id_API']
for c in col:
    augmented_df2[c] = augmented_df2.apply(assign_value, column = c, axis = 1)

In [50]:
augmented_df2.drop(columns = augmented_df2.filter(regex = '._added').columns, inplace=True) #drop _added columns
augmented_df2.drop(columns = ['copy_index', 'copy_index_2'], inplace = True) #drop las columnas con indices

In [51]:
augmented_df2.sample(5)

,anio_ing,nacionalidad,provincia_part,loc_part,cp_part,user_id,provincia_por_cp,localidad_por_cp,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
copy_index,,,,,,,,,,,,,,,,,,,,,,,,,
61624,2018,UY,CF,CABA,1225,ID_61624,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,02042010001,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
19356,2015,AR,BA,LOMAS DEL MIRADOR,1752,ID_19356,BUENOS AIRES,NO UNICA LOCALIDAD,None,LOMAS DEL MIRADOR,06427010007,LA MATANZA,60427,LA MATANZA,6427,BUENOS AIRES,6,-34.6657,-58.5332,-34.770165,-58.625449,-34.770165,-58.625449,-36.676942,-60.558832
59905,2017,DESCONOCIDO,NaN,CDAD AUTONOMA BS AS,0,ID_59905,CP INDEFINIDO,CP INDEFINIDO,None,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NaN,NaN
71692,2018,AR,CF,CABA,1424,ID_71692,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,02042010001,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
34566,2016,AR,CF,CABA,1407,ID_34566,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,02042010001,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856


## EDA

In [52]:
augmented_df2[augmented_df2['localidad_por_API'] == 'NO HUBO REQUEST'].shape

(6877, 25)

In [53]:
augmented_df2.shape

(87402, 25)

In [54]:
augmented_df2[augmented_df2['localidad_por_API'] == 'NO HUBO REQUEST'].shape[0]/augmented_df2.shape[0] * 100

7.868241001350083

## Analisis de los datos

Aproximadamente un 7.8% no pudo ser identificado.<br>
Tenemos que mencionar que confiar en el codigo postal lejos esta de ser el mejor de los casos (los sujetos pueden olvidarlo o ingresarlo erroneamente sin darse cuenta) pero frente los inconvenientes debidos a la falta de normalizacion de la base inicial nos parecio la mejor opcion. 
Hay que aclarar que otro problema es que los usuarios de CABA usualmente no ponen la localidad en el campo `loc_part` por lo que deberiamos hacer un sistema doble en el cual se asigne el campo de `provincia_por_cp` a `provincia_por_API`

---

# Modificacion de anomalias

In [55]:
augmented_df2[augmented_df2.localidad_por_API == 'CABALLITO'].sample(10) 
#  Debo arreglar que todos los que dice CABA y no tengan localidad por CP pasen a ser solo Capital

,anio_ing,nacionalidad,provincia_part,loc_part,cp_part,user_id,provincia_por_cp,localidad_por_cp,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
copy_index,,,,,,,,,,,,,,,,,,,,,,,,,
33258,2016,SE,CF,CABA,1419,ID_33258,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,02042010001,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
61111,2018,AR,CF,CABA,1424,ID_61111,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,02042010001,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
66775,2018,AR,CF,CABA,1431,ID_66775,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,02042010001,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
72372,2018,AR,CF,CABA,1405,ID_72372,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,02042010001,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
27316,2016,PY,CF,CABA,1426,ID_27316,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,02042010001,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
30674,2016,AR,CF,CABA,1417,ID_30674,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,02042010001,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
70882,2018,AR,CF,CABA,1428,ID_70882,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,02042010001,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
73437,2018,AR,CF,CABA,1184,ID_73437,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,02042010001,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
11719,2014,AR,CF,CABA,1417,ID_11719,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,02042010001,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856


In [56]:
mask = augmented_df2[(augmented_df2.loc_part == 'CABA')  &
              (augmented_df2.localidad_por_cp == 'NO UNICA LOCALIDAD') &
              (augmented_df2.provincia_por_cp == 'CIUDAD AUTONOMA DE BUENOS AIRES')].index

In [57]:
cols_modif = [9,10,11,12,13,14,17,18,19,20,21,22]
augmented_df2.iloc[mask,cols_modif] = 'NO HUBO REQUEST'

In [58]:
#  Debo arreglar que todos los que dicen `CDA AUTONOMA DE BS AS` o tengan localidad por CP pasen a ser solo Capital
mask = augmented_df2[(augmented_df2.loc_part == 'CDAD AUTONOMA BS AS') &
              (augmented_df2.localidad_por_cp == 'NO UNICA LOCALIDAD') &
              (augmented_df2.provincia_por_cp == 'CIUDAD AUTONOMA DE BUENOS AIRES')].index

augmented_df2.loc[mask,'localidad_id_API'] = 'NO HUBO REQUEST'
augmented_df2.loc[mask,'provincia_por_API'] = 'CIUDAD AUTÓNOMA DE BUENOS AIRES'
augmented_df2.loc[mask,'provincia_id_API'] = 2
augmented_df2.loc[mask,'centroide_lat_provincia'] = -34.614493
augmented_df2.loc[mask,'centroide_lon_provincia'] = -58.445856

In [59]:
augmented_df2

,anio_ing,nacionalidad,provincia_part,loc_part,cp_part,user_id,provincia_por_cp,localidad_por_cp,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
copy_index,,,,,,,,,,,,,,,,,,,,,,,,,
0,2014,AR,CF,CABA,1416,ID_0,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,-34.614493,-58.445856
1,2014,AR,CF,CABA,1416,ID_1,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,-34.614493,-58.445856
2,2014,AR,BA,HURLINGHAM,1686,ID_2,BUENOS AIRES,HURLINGHAM,None,HURLINGHAM,06408010001,HURLINGHAM,60408,HURLINGHAM,6408,BUENOS AIRES,6,-34.5931,-58.6357,-34.5992,-58.6497,-34.5992,-58.6497,-36.676942,-60.558832
3,2014,AR,BA,ZARATE,2800,ID_3,BUENOS AIRES,NO UNICA LOCALIDAD,None,ZARATE,06882050000,ZÁRATE,60882,ZÁRATE,6882,BUENOS AIRES,6,-34.0999,-59.0246,-33.9971,-59.1285,-33.9971,-59.1285,-36.676942,-60.558832
4,2014,AR,CF,CABA,1425,ID_4,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,-34.614493,-58.445856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87397,2019,AR,CF,CDAD AUTONOMA BS AS,1417,ID_87397,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,-34.614493,-58.445856
87398,2019,AR,CF,CDAD AUTONOMA BS AS,1182,ID_87398,CIUDAD AUTONOMA DE BUENOS AIRES,PALERMO,COMUNA 14,PALERMO,02098010001,NO HUBO REQUEST,NaN,COMUNA 14,2098,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.5739,-58.4224,NaN,NaN,-34.5738,-58.4222,-34.614493,-58.445856
87399,2019,AR,CF,CDAD AUTONOMA BS AS,1440,ID_87399,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,-34.614493,-58.445856


# Analisis de los datos post modificacion

In [64]:
print(augmented_df2[augmented_df2.provincia_por_API == 'NO HUBO REQUEST'].shape[0] / augmented_df2.shape[0] * 100)
print(augmented_df2[augmented_df2.localidad_por_API == 'NO HUBO REQUEST'].shape[0] / augmented_df2.shape[0] * 100)
print(augmented_df2[augmented_df2.provincia_por_API == 'CIUDAD AUTÓNOMA DE BUENOS AIRES'].shape[0] / augmented_df2.shape[0] * 100)

print(augmented_df2[(augmented_df2.localidad_por_API == 'NO HUBO REQUEST') &
                    (augmented_df2.provincia_por_API != 'CIUDAD AUTÓNOMA DE BUENOS AIRES')].shape[0] / augmented_df2.shape[0] * 100)

print(augmented_df2[(augmented_df2.provincia_por_API == 'CIUDAD AUTÓNOMA DE BUENOS AIRES') & (augmented_df2.localidad_por_API != 'NO HUBO REQUEST')].shape[0] /
      augmented_df2[augmented_df2.provincia_por_API == 'CIUDAD AUTÓNOMA DE BUENOS AIRES'].shape[0] * 100)

4.104025079517632
44.384567858859064
45.168302784833294
4.104025079517632
10.821216880287755


Posterior a las modificaciones solo el 4% del padron no tiene un campo para provincia. <br>
Un 44.3 % no tiene una localidad definida. De ese 44% la mayoria responde a personas de CABA (un 45.1 % del padron). Si excluimos a esta gente solo un 4.1 % del padron no tiene una localidad normalizada. <br>
Por otro lado, los usuarios de CABA que SI POSEEN localidad definida son solo el 10.8 % del padron de CABA


In [67]:
pickle.dump(augmented_df2, open('HI_localizacion_socios.pickle','wb'))

---

# Analisis de la implementacion de un algoritmo para geolocalizar usuarios

In [28]:
augmented_df2 = pickle.load(open('./HI_localizacion_socios.pickle', 'rb'))

## Comparacion por provincia

In [32]:
df_clean.isna().sum().provincia_part / df_clean.shape[0] * 100

5.512459669115123

In [45]:
# Total de Nan y `NO HUBO REQUEST` en tabla aumentada
total_miss = augmented_df2.provincia_por_API.isna().sum() + augmented_df2[augmented_df2.provincia_por_API == 'NO HUBO REQUEST'].count().provincia_por_API
total_miss / augmented_df2.shape[0] * 100

4.333996933708611

In [47]:
# Provincias nuevas incluidas
df_clean.isna().sum().provincia_part - total_miss

1030

Mientras que nuestro dataset inicial contaba con un 5.51% de valores missing para el campo de provincia, tras correr el algoritmo se alcanzo se paso a 4.33% de missing values. En otras palabras, tras correr el algoritmo se obtuvo un incremento de **1030** usuarios con un campo de provincia asignado.

## Comparacion por localidad

In [55]:
df_clean.isna().sum().loc_part / df_clean.shape[0] * 100

0.30090844603098327

In [56]:
(augmented_df2.localidad_por_API.isna().sum() +  augmented_df2[augmented_df2.localidad_por_API == 'NO HUBO REQUEST'].shape[0]) / df_clean.shape[0] * 100

44.61453971305004

A pesar de que solo el 0.3 % no presenta una localidad reportada, al intentar localizar la localidad usando nuestro algoritmo vemos que un 44% de los usuarios no tienen una localidad definida, normalizada y geolocalizada (con un centroide). 

In [67]:
print(f'Cantidad de usuarios de la provincia de Buenos Aires por API: {augmented_df2[(augmented_df2.provincia_por_API == "BUENOS AIRES")].shape[0]}')

Cantidad de usuarios de la provincia de Buenos Aires por API: 42447


In [77]:
print(f'Cantidad de usuarios de la provincia de Buenos Aires por API sin localidad asignada: {augmented_df2[(augmented_df2.localidad_por_API == "NO HUBO REQUEST") & (augmented_df2.provincia_por_API == "BUENOS AIRES")].shape[0]}')

Cantidad de usuarios de la provincia de Buenos Aires por API sin localidad asignada: 0


In [82]:
print(f"Cantidad de usuarios de CABA por API: {augmented_df2[augmented_df2.provincia_por_API == 'CIUDAD AUTÓNOMA DE BUENOS AIRES'].shape[0]}")

Cantidad de usuarios de CABA por API: 39478


In [65]:
print(f"Cantidad de usuarios de CABA por API sin localidad asignada: {augmented_df2[(augmented_df2.localidad_por_API == 'NO HUBO REQUEST') & (augmented_df2.provincia_por_API == 'CIUDAD AUTÓNOMA DE BUENOS AIRES')].shape[0]}")

Cantidad de usuarios de CABA por API sin localidad asignada: 35206


In [91]:
prom_miss_caba = augmented_df2[(augmented_df2.localidad_por_API == 'NO HUBO REQUEST') &
                               (augmented_df2.provincia_por_API == 'CIUDAD AUTÓNOMA DE BUENOS AIRES')].shape[0] * 100 \
                 / augmented_df2[(augmented_df2.provincia_por_API == 'CIUDAD AUTÓNOMA DE BUENOS AIRES')].shape[0] 

In [94]:
100 - prom_miss_caba

10.82121688028775

In [73]:
total_con_loc = augmented_df2[(augmented_df2.localidad_por_API.notna()) & (augmented_df2.localidad_por_API != 'NO HUBO REQUEST')].shape[0]
(augmented_df2.shape[0] - total_con_loc) / augmented_df2.shape[0] * 100

44.61453971305004

**Todos** los usuarios de la Provincia de **Buenos Aires (100%)** se les asigno una localidad normalizada y se los georeferencio (42447 usuarios).<br>
De los usuarios de la **Capital Federal** (39478 usuarios) solo se pudo asignar una localidad y geolocalizar al **10.8%** (4272 usuarios) <br>
En **total** un **44.6%** de los usuarios (48408) fueron **geolocalizados** y sus **localidades** fueron **normalizadas**.